In [1]:
import requests
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from dash import Dash, dcc, html, Input, Output

In [2]:
# Step 1: Fetch data from DeFiLlama API
def fetch_data():
    url = 'https://api.llama.fi/v2/chains'
    response = requests.get(url)
    if response.status_code == 200: # data was fetched successfully
        data = response.json()
        return pd.DataFrame(data)
    else:
        raise Exception('Error fetching data from DeFiLlama API')

def fetch_historical_tvl(chain_name):
    url = f'https://api.llama.fi/v2/historicalChainTvl/{chain_name}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)  # Directly create DataFrame from the list of dictionaries
        df['date'] = pd.to_datetime(df['date'], unit='s')
        return df.tail(120)  # Get the last 120 days
    else:
        raise Exception(f'Error fetching historical TVL data for {chain_name}')

# Step 2: Prepare the data
def prepare_data(df):
    # Keep only name and tvl
    df = df[['name', 'tvl']]
    
    # Sort the data to get the top 10 chains by TVL
    df = df.sort_values(by='tvl', ascending=False).head(10)
    
    return df



In [3]:
# Step 3: Create the Dash app and the layout
def create_dashboard(df):
    app = Dash(__name__)
    app.layout = html.Div([
        html.H1('Top 10 Chains Dashboard'),
        dcc.Dropdown(
            id='metric-dropdown',
            options=[
                {'label': 'Total Value Locked (TVL)', 'value': 'tvl'}
            ],
            value='tvl'
        ),
        dcc.Graph(id='bar-chart'),
        dcc.Graph(id='historical-tvl-chart')
    ])

    # Step 4: Add callback to update the chart based on selected metric
    @app.callback(
        [Output('bar-chart', 'figure'),
         Output('historical-tvl-chart', 'figure')],
        [Input('metric-dropdown', 'value')]
    )
    def update_charts(selected_metric):
        # Bar Chart for selected metric
        bar_fig = px.bar(df, x='name', y=selected_metric, title=f'Top 10 Chains by {selected_metric.upper()}')
        
        # Line Chart for historical TVL of top 10 chains
        historical_tvl_fig = go.Figure()
        for chain_name in df['name']:
            historical_tvl_df = fetch_historical_tvl(chain_name)
            historical_tvl_fig.add_trace(go.Scatter(
                x=historical_tvl_df['date'],
                y=historical_tvl_df['tvl'],
                mode='lines',
                name=chain_name
            ))
        historical_tvl_fig.update_layout(title='Historical TVL of Top 10 Chains (Last 120 Days)')
        
        return bar_fig, historical_tvl_fig

    return app

In [4]:
# Step 5: Run the app if the script is executed directly
if __name__ == '__main__':
    df = fetch_data()
    df = prepare_data(df)
    app = create_dashboard(df)
    app.run_server(debug=True, use_reloader=False)  # Disable reloader to prevent multiple fetches of data